<a href="https://colab.research.google.com/github/hmgu-itg/VolosSummerSchool/blob/master/VSS_2024/5_Workshop_molQTL/5_Workshop_molQTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QTL practical
*(Mauro Tutino, mauro.tutino@helmholtz-munich.de)*

-   In this workshop, we will familiarize ourselves with the basic implementation of QTL mapping. For the purposes of this practical we will use genotype and gene expression data in human individuals.

-   **Note!** The data have already passed the necessary QC (Quality Control checks e.g.normalization) and are ready to be analysed.

*First we have to install several packages...*

## Google Drive

Let us set up the connection with Google Drive. We need the kernel to be set to python. You can do this inside "change runtime type" and selecting python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Since the whole notebook is in R, let's change the kernel to be in R. You can do this inside "change runtime type" and selecting R

In [ ]:
# If you did the above, this is not needed anymore
#!pip install "pandas<2.0.0"
#!pip install rpy2
#import os # python related package to list files in the defined directory
#import rpy2.ipython # allows to execute R if necessary from a python notebook
#%load_ext rpy2.ipython

In [ ]:
# Install all the necessary packages.
if (!requireNamespace('dplyr', quietly = TRUE))
install.packages('dplyr')
if (!requireNamespace('tidyr', quietly = TRUE))
install.packages('tidyr')
if (!requireNamespace('ggplot2', quietly = TRUE))
install.packages('ggplot2')
if (!requireNamespace('magrittr', quietly = TRUE))
install.packages('magrittr')
#install.packages('factoextra')
#if (!requireNamespace('devtools', quietly = TRUE))
#install.packages('devtools')
if (!requireNamespace('IRdisplay', quietly = TRUE))
install.packages('IRdisplay')
if (!requireNamespace('MatrixEQTL', quietly = TRUE))
install.packages('MatrixEQTL')
if (!requireNamespace('corrplot', quietly = TRUE))
install.packages("corrplot")

In [ ]:
# Load the libraries.
library(dplyr)
library(tidyr)
library(ggplot2)
library(magrittr)
library(IRdisplay)
library(MatrixEQTL)
library(corrplot)

In [ ]:
setwd("/content/drive/My Drive/Complex_Traits")

In [ ]:
list.files()

Change the working directory to where the "Data_and_dependencies.zip" file is

In [ ]:
setwd('5_Workshop_molQTL')

# R trick, you can run bash commands (or any other command line code) from within R using the function 'system'

In [ ]:
# Unzip the Data & dependencies.
system('unzip Data_and_dependencies.zip', intern = T)


In [ ]:
list.files('Data_and_dependencies')


- A quick overview of the things we are going to cover.

In [ ]:
setwd('Data_and_dependencies') # Set the working directory
getwd()

display_png(file = 'workflow.png')

---
# Part 1. Getting to know your data

-   First, we are going to load all the necessary files:
  - What we need :
      - a.  Genotypes
      - b.  Gene Expression data
      - c.  Info for Genes and SNPs (position, chromosome etc.)

In [ ]:
#.a
snps = read.table('input/uniq.snps.txt', header = T) # Genotype info - it has been converted to dosage matrix.

snps <- snps %>%
        select(snp, everything()) # Re-arrange the columns.
head(snps)
dim(snps)

In [ ]:
# b.
ge = read.table('input/ge.profile.txt', header = T) # Gene expression data (RPKM values).
rownames(ge) <- ge$IID
head(ge)

In [ ]:
# c.
snp.loc = read.table('input/snps.loc.txt', header = T) # Location info for SNPs.
head(snp.loc)
gene.loc = read.table('input/gene.loc.txt', header = T) # Location info for Genes.
head(gene.loc)

---
# Questions Part 1.

  1.  How many Samples, SNPs and Genes do we have ?
  2.  On which chromosome are the SNPs and Genes located ?

In [ ]:
#@title # 1. How many Samples, SNPs and Genes do we have?
cat('Number of Genes:', ncol(ge)-1, '\n' );
cat('Number of samples:', ncol(snps)-1, '\n');
cat('Number of SNPs:', nrow(snps), '\n')

In [ ]:
# 1. TYPE YOUR RESPONSE HERE

In [ ]:
# @title # 2. On which chromosome are the SNPs and Genes located ?
cat('Chromosome of SNPs:', unique(snp.loc$chr), '\n')
cat('Chromosome of Genes:', unique(gene.loc$chr), '\n')

In [ ]:
# 2. TYPE YOUR RESPONSE HERE

---
# Part 2. Exploratory data analysis and covariate selection

- Although the data have passed the necessary QC steps, we will check for  outlier individuals/observations for both Genotype and GE data.
A common practice to do this -in high dimensional data- is via Principal Component Analysis (PCA).

*-Outlier observations can inflate false positive findings.*

- We will start with the PCA of genotypes. After computing the Principal Components (PCs) we are going to plot the first two of them.

*-By definition the first PC captures the majority of variance in the variables of interest.*

NOTE: here we use the R function prcomp() to calculate the PCs. Usually this is done with specialised software such as Plink

In [ ]:
# PCA for genotypes via prcomp in R.
geno.pca <- prcomp(t(snps[,-1]), scale. = T) # Remember to include only the variables. NB we have to transpose the data frame since the package needs the variables as columns and subjects as rows!
summary(geno.pca) # Take a look at the output

# Visual inspection of PCs, plotting the first 2 PCs.
pca.data <-  data.frame(PC1=geno.pca$x[,1], PC2=geno.pca$x[,2])

cat('', '\n')
PC1.sd <-  sd(pca.data$PC1)
PC2.sd <-  sd(pca.data$PC2)

ggplot(data=pca.data, aes(x=PC1, y=PC2)) +
  geom_point() +
  geom_vline(xintercept = mean(pca.data$PC1) - 3*PC1.sd, colour="red", linetype = "longdash") +
  geom_vline(xintercept = mean(pca.data$PC1) + 3*PC1.sd, colour="red", linetype = "longdash") +
  geom_hline(yintercept = mean(pca.data$PC2) - 3*PC2.sd, colour="blue", linetype = "longdash") +
  geom_hline(yintercept = mean(pca.data$PC2) + 3*PC2.sd, colour="blue", linetype = "longdash") +
  ggtitle('PCA for Genotype data')



The points in the scatter plot are randomly scattered, no indication of clustering.

In [ ]:
# Here we can access the Principal Component scores for each individual
head(geno.pca$x)

Next, we are going to produce the same plot for the GE data...

In [ ]:
# The same procedure as above.
ge.pca = prcomp(ge[,-1], scale = T) # No need to transpose the data frame.
summary(ge.pca)

# PCA data.frame
pca.data = data.frame(PC1=ge.pca$x[,1], PC2=ge.pca$x[,2])

PC1.sd = sd(pca.data$PC1)
PC2.sd = sd(pca.data$PC2)

# Visual inspection of PCs.
ggplot(data=pca.data, aes(x=PC1, y=PC2)) +
  geom_point() +
  geom_vline(xintercept = mean(pca.data$PC1) - 3*PC1.sd, colour="red", linetype = "longdash") +
  geom_vline(xintercept = mean(pca.data$PC1) + 3*PC1.sd, colour="red", linetype = "longdash") +
  geom_hline(yintercept = mean(pca.data$PC2) - 3*PC2.sd, colour="blue", linetype = "longdash") +
  geom_hline(yintercept = mean(pca.data$PC2) + 3*PC2.sd, colour="blue", linetype = "longdash") +
  ggtitle('PCA for GE data')

# Variation explained for each PC.
pca.object.var <- summary(ge.pca)
pca.object.var.per <- as.numeric(pca.object.var$importance[2,])*100

barplot(pca.object.var.per[1:10], main="Percent variation for PC1-10",
        xlab = "PC",
        ylab = "Percent Variation (%)",
        xlim = c(0, 11),
        ylim = c(0,100))



# Questions Part 2.

  1. What do you think of the PCA plot in GE data ? Any outliers ?
  2. How much variation do the first 3 components explain in GE data ?


In [ ]:
# @title 1. What do you think of the PCA plot in GE data ? Any outliers ?

# As we can see there is one point that falls outside the bondaries but it is still very close.
# Even if we are going to exclude this observation it is unlikely that this will influence the behaviour of the other points.


######.1 TYPE YOUR RESPONSE HERE (text only)

In [ ]:
#@title #### 2. How much variation do the first 3 components explain in GE data ?

pca.object.var$importance[3,3]*100


- Except for outlier detection, PCA, can also be used for:
    - covariate selection
    - account for unknown covariates

*-Recall from the previous lectures that when we are modelling the effect of a variable on an outcome of interest, in our case the genetic effect of SNP on GE, we have to control for additional factors that may confound the relationship that we are investigating.*

- We have data for Age, Sex and BMI for each participant, go ahead and load the data.

In [ ]:
# Load the covariates.
age = read.table('input/Age.txt', header = F)
age = age[,c(1,3)] # Drop the duplicated column.
colnames(age) <- c('IID','age')

sex = read.table('input/Sex.txt', header = F)
sex = sex[,c(1,3)] # Drop the duplicated column.
colnames(sex) <- c('IID','sex')

bmi = read.table('input/BMI.txt', header = F)
bmi = bmi[,c(1,3)] # Drop the duplicated column.
colnames(bmi) <- c('IID','bmi')

In [ ]:
# Correlation plot of PCs with covariates

# Do the the PCs of GE correlate with the observed covariates/factors ?
metadata = merge(age, sex, by = 'IID')
metadata = merge(metadata, bmi, by = 'IID')

# Make sure that we have the same order in SampleIDs between dataframes.
metadata <- metadata[match(ge$IID, metadata$IID),] # Tricky the first argument of match defines the order of matching values.

identical(ge$IID, metadata$IID) # Sanity check.

# Data manipulation to meet the need of PCAtools.
t.ge <- t(as.matrix(ge))
colnames(t.ge) <- t.ge[1,]
t.ge <- t.ge[-1,]
rownames(t.ge) <- NULL
t.ge <- data.frame(t.ge)
t.ge[,] <- sapply(t.ge, as.numeric)

rownames(metadata) <- metadata$IID

ge.pca_3PC <- as.data.frame(ge.pca$x[,c(1:3)])
ge.pca_3PC$IID <- rownames(ge.pca_3PC)

metadata <- merge(metadata, ge.pca_3PC, by = 'IID')
metadata$sex <- as.numeric(as.factor(metadata$sex)) # convert sex to numerical for correlation

head(metadata)

# Calculate correlation.
# In this case, we are not interested in the correlation between PCs
cor.res <- cor(metadata[,2:4], metadata[,5:7])

# Plot correlation
corrplot(cor.res, type = "upper", order = "original",
         tl.col = "black", tl.srt = 45, addCoef.col = 'black')

# Positive correlations are displayed in blue and negative correlations in red color.
# Color intensity and the size of the circle are proportional to the correlation coefficients.
# In the right side of the correlogram, the legend color shows the correlation coefficients and the corresponding colors.

- Only PC3 shows a moderate correlation with sex

---
# Part 3. Data formatting for MatrixEQTL

MatrixEQTL is a tool specialized for genetic association analyses with molecular traits (Gene Expression data, Metabolites etc.). Instead of doing single-point associations, like for GWAS tools, it performs matrix operations, which allows to run thousands of associations at once. It also allows to run cis and trans QTL separately.


In this section we are going to prepare the input files for MatrixEQTL


In [ ]:
# Format the gene expression data - samples need to be on columns
dim(ge)
print("Before")
ge[1:5,1:5]

ge <- t(ge)

colnames(ge) <- ge[1,]
ge <- ge[-1,]
print("After")
ge[1:5,1:5]

In [ ]:
# Format genotype data - samples need to be on columns
dim(snps)
print("Before")
snps[1:5,1:5]

rownames(snps) <- snps[,1]
snps <- snps[,-1]
print("After")
snps[1:5,1:5]


In [ ]:
# Format covariates data - samples need to be on columns

covariates = merge(age, sex, by = 'IID')
covariates = merge(covariates, bmi, by = 'IID')

# we need to convert sex to numeric, F==0 and M==1
covariates <- covariates %>%
                mutate(sex = case_when(
                sex == "F" ~ "0",
                sex == "M" ~ "1",
                TRUE ~ sex),
                sex = as.numeric(sex)) %>% # Add the first three genotype principal components
                left_join(.,
                         as.data.frame(geno.pca$x) %>%
                          dplyr::select(1:3) %>%
                          mutate(IID = rownames(.)),
                         by='IID')



# Transpose and format
covariates <- t(covariates)
colnames(covariates) <- covariates[1,]
covariates <- covariates[-1,]

covariates[,1:5]


In [ ]:
# Check that the samples are in the right order between the files
index <- colnames(snps)

ge <- ge[,match(index, colnames(ge))]
covariates <- covariates[,match(index, colnames(covariates))]

ge[1:5,1:5]
covariates[,1:5]
snps[1:5,1:5]

identical(colnames(ge), colnames(snps))
identical(colnames(ge), colnames(covariates))


In [ ]:
# write files to disk
write.table(ge, file="input/ge_matrixEQTL.txt", quote = FALSE, sep = "\t", row.names = TRUE, col.names = TRUE)
write.table(snps, file="input/snps_matrixEQTL.txt", quote = FALSE, sep = "\t", row.names = TRUE, col.names = TRUE)
write.table(covariates, file="input/covariates_matrixEQTL.txt", quote = FALSE, sep = "\t", row.names = TRUE, col.names = TRUE)



In [ ]:
# we need to provide the snp and gene positions in the format snp	chr	pos and geneid	chr	s1	s2
snp.loc = read.table('input/snps.loc.txt', header = TRUE, ) # Location info for SNPs.
head(snp.loc)
gene.loc = read.table('input/gene.loc.txt', header = TRUE) # Location info for Genes.
head(gene.loc)

dim(snp.loc)
dim(snps)

In [ ]:
# @title Find the Bug, DO NOT RUN JUST YET AND DO NOT PEEK. We will run this one at the end
snp.loc$chr <- paste0("chr",snp.loc$chr)

---
# Part 4. Association analysis - QTL mapping
- We are going to perform a cis-association analysis on chromosome1. That is, only the variants that fall inside a +/- 1Mb window from the TSS of the gene of interest are going to be tested.

  - Recall that we are using a linear regression model to estimate the genetic effect of each variant for the expression of each Gene.



In [ ]:
## Genotype file name
SNP_file_name = "input/snps_matrixEQTL.txt" # dosage of alt allele. Samples on columns

### Gene expression file name
expression_file_name = "input/ge_matrixEQTL.txt"

## Covariates file name
covariates_file_name = "input/covariates_matrixEQTL.txt"




In [ ]:
######----------------------------- set up parameters

# Error covariance matrix
# Set to numeric() for identity.
errorCovariance = numeric();

# Output file names
transFileName = "eQTL_trans.txt"
cisFileName = "eQTL_cis.txt"

# Distance for local gene-SNP pairs
cisDist = 1e6

# model to use: modelANOVA, modelLINEAR, or modelLINEAR_CROSS
useModel = modelLINEAR

cis.thresh = 5e-8 # Save cis results with p-value < 5e-8
trans.thresh = 0 # Don't perform trans eQTL analysis. Change this to anything > 0 if you want to perfom trans analysis. NB it is lots of tests!



In [ ]:
######----------------------------- load files

## Load genotype data
geno = SlicedData$new();
geno$fileDelimiter = "\t";      # the TAB character
geno$fileOmitCharacters = "NA"; # denote missing values;
geno$fileSkipRows = 1;          # one row of column labels
geno$fileSkipColumns = 1;       # one column of row labels
geno$fileSliceSize = 2000;      # read file in slices of 5,000 rows
geno$LoadFile(SNP_file_name);


## Load gene expression data
gene = SlicedData$new();
gene$fileDelimiter = "\t";      # the tab character
gene$fileOmitCharacters = "NA"; # denote missing values;
gene$fileSkipRows = 1;          # one row of column labels
gene$fileSkipColumns = 1;       # one column of row labels
gene$fileSliceSize = 2000;      # read file in slices of 2,000 rows
gene$LoadFile(expression_file_name);

## Load covariates
cvrt = SlicedData$new();
cvrt$fileDelimiter = "\t";      # the comma character
cvrt$fileOmitCharacters = "NA"; # denote missing values;
cvrt$fileSkipRows = 1;          # one row of column labels
cvrt$fileSkipColumns = 1;       # one column of row labels
#If loading from file
if(length(covariates_file_name)>0) {
 cvrt$LoadFile(covariates_file_name);
}



In [ ]:
#######---------- run matrixeQTL
me = Matrix_eQTL_main(
    snps = geno,
    gene = gene,
    cvrt = cvrt,
    output_file_name     = file.path("output", transFileName),
    output_file_name.cis = file.path("output", cisFileName),
    pvOutputThreshold     = trans.thresh,
    pvOutputThreshold.cis = cis.thresh,
    useModel = useModel,
    errorCovariance = numeric(),
    verbose = FALSE,
    snpspos = snp.loc,
    genepos = gene.loc,
    cisDist = cisDist,
    pvalue.hist = "qqplot",
    min.pv.by.genesnp = TRUE,
    noFDRsaveMemory = TRUE);




In [ ]:
# MatrixEQTL saved the cis eQTL summary statistics to disk but it also saved important infomation in the "me" variable
# what class variable is it?
# Check what values are stored in there
class(me)
names(me)
names(me$cis)

# Make Q-Q plot
plot(me)

---
# Questions Part 4.

  1. How many tests did we perform ? How many eGenes (Genes with at least one sig. SNP) and eQTLs do we have ? NOTE: here we consider a p-value < 5e-8 as statistically significant.
  2. Retrieve the best eQTL per Gene.
  3. Imagine we set the parameter trans.thresh = 1, i.e.  save all trans eQTL results, how many trans eQTL test would have MatrixEQTL performed?

In [ ]:
# @title 1. How many tests did we perform ? How many eGenes (Genes with at least one sig. SNP) and eQTLs do we have ? NOTE: here we consider a p-value < 5e-8 as statistically significant.
paste('Number of cis-tests: ', me$cis$ntests)
paste('Number of eGenes:', sum(me$cis$min.pv.gene < 5e-8))
paste('Number of eqtls: ', me$cis$neqtls)


In [ ]:
# 1. TYPE YOUR RESPONSE HERE

In [ ]:
# @title 2. Manually retrieve the best (most significant) eQTL per Gene from the matrixeQTL output and store it in a object called "eqtls"

# Manually extract the minimum p-value for each gene
eqtls <- read.table(file.path("output", cisFileName), header = T)

eqtls %>%
  group_by(gene) %>%
  slice(which.min(p.value))


In [ ]:
# 2. TYPE YOUR RESPONSE HERE

In [ ]:
# @title 3. Imagine we set the parameter trans.thresh = 1, i.e.  save all trans eQTL results, how many trans eQTL test would have MatrixEQTL performed?

paste('Number of trans-tests:', nrow(snps)*nrow(ge))


In [ ]:
# 3. TYPE YOUR RESPONSE HERE


In [ ]:
# Do the numbers match the ones reported by matrixEQTL?

paste('Number of eGenes: ', length(unique(eqtls$gene)))
paste('Number of eqtls: ', nrow(eqtls))


In [ ]:
# Why do you think the following result gives a different number of eQTLs?
paste('Number of eqtls: ', nrow(eqtls))
paste('Number of eqtls: ', sum(me$cis$min.pv.snps < 5e-8))


In [ ]:
head(eqtls)
signifSNP <- eqtls %>%
    dplyr::filter(p.value < 5e-8) %>%
    pull(SNP)

paste('N. duplicated SNPs: ', sum(duplicated(signifSNP)))

---
# Further exploration


- Next, make a histogram for the distance of cis-eQTLs from the TSS of the canditate eGene.

In [ ]:
# Distance to TSS.
eqtls.tss <- eqtls %>%
                left_join(., snp.loc[,-2], by=c("SNP"="SNP_ID")) %>% # merge MatrixEQTL summary stats with SNP location
                rename("SNP_pos"="pos") %>%
                left_join(., gene.loc[,c(1,3)], by="gene") %>% # now also add gene location
                rename("gene_pos"="start") %>%
                mutate(dist.to.tss=gene_pos-SNP_pos) # calculate TSS distance
head(eqtls.tss)

hist(eqtls.tss$dist.to.tss, breaks = 100, main = 'Histogram of cis-eQTL distances ' , xlab = 'Distance to TSS')
abline(v=0)


As expected, most of the signals are clustereed around the TSS of the Gene.


Now, let's retrieve the best eQTL (the one with the smallest p-value) for the eGene with the most findings.

In [ ]:
# Best eQTL for the Gene with most findings.
eqtls %>%
    group_by(gene) %>%
    summarise(n.eqtls = n()) %>%
    arrange(desc(n.eqtls))



In [ ]:
top.signal <- eqtls %>%
                dplyr::filter(gene == 'ENSG00000198468.3') %>%
                slice(which.min(p.value))
head(top.signal)

- Visualize the effect of the best eQTL for the ENSG00000198468.3 Gene.

In [ ]:
# Boxplot for the best eQTL.
plot.data <- snps %>%
                dplyr::filter(rownames(snps) == top.signal$SNP)

plot.data = as.data.frame(t(plot.data))

plot.data = cbind(plot.data, ge[rownames(ge) == top.signal$gene,]) # add gene expression


boxplot(as.numeric(plot.data[[2]]) ~ as.factor(as.character(plot.data[[1]])), xlab = "1_213014465_G_A",
   ylab = "GE", main = "ENSG00000198468.3")

---


In [ ]:
# @title Find the bug solution (kind of)
head(snp.loc)
head(gene.loc)